In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import configparser
sys.path.insert(0, 'Utils/')

from AISDataManager import AISDataManager
import SimpleUtils as sU
import Constants as c
import TimeUtils as timeUtils
import HMUtils as hMUtil
import GeoCompute as gC

aISDM = AISDataManager()

config = configparser.ConfigParser()
config.read('DefaultConfig.INI')

['DefaultConfig.INI']

In [3]:
lonMin = (float)(config['LSTM_MULTI_TASK_TRAIN']['LON_MIN'])
lonMax = (float)(config['LSTM_MULTI_TASK_TRAIN']['LON_MAX'])

latMin = (float)(config['LSTM_MULTI_TASK_TRAIN']['LAT_MIN'])
latMax = (float)(config['LSTM_MULTI_TASK_TRAIN']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

-122.0 33.4
-118.5 36.4


In [4]:
class VesselTypeSource:
    """
    The VesselTypeSource object contains information. 
    which is useful for the training data
    """
    def __init__(self, srcDir, typeVes):
        self.srcDir = srcDir
        self.type = typeVes
        self.trainList = []

In [5]:
sourceDir1 = config['LSTM_MULTI_TASK_TRAIN']['SRC_DIR_1']
sourceDir2 = config['LSTM_MULTI_TASK_TRAIN']['SRC_DIR_2']

modelDir = config['LSTM_MULTI_TASK_TRAIN']['MODEL_DIR']
prevTS = int(config['LSTM_MULTI_TASK_TRAIN']['PREV_TS'])

print(sourceDir1)
print(sourceDir2)
print(modelDir)
print(prevTS)

/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/MMSI_1004_TRAIN_INTP/
/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/MMSI_1024_TRAIN_INTP/
/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/Models/TS_5/
12


In [6]:
vesselSource1 = VesselTypeSource(sourceDir1,0)
vesselSource2 = VesselTypeSource(sourceDir2,1)

vesselDataSources = [vesselSource1, vesselSource2]
print(vesselDataSources[0])
print(vesselDataSources[1])

In [7]:
def get_traj_data(fileName):
    """
    """
    sourceDF,_ = aISDM.load_data_from_csv(fileName)
    return sourceDF.to_numpy()

In [8]:
def convert_seq_to_x_y(seq, vType, prevTimeStamp):
    """
    """
    #-prevTimeStamp is is to take care of boundary condition
    #since we are considering prevTimeStamp time stamps for the input data
    xNumRows = seq[:-(prevTimeStamp),:].shape[0]
    print(xNumRows)
    dateStr = seq[0][0]
    monF = int(dateStr.split("-")[1])
    print(monF)
    monData = np.zeros((xNumRows, 12))
    monData[:,(12-monF)] = 1
    
    typeArr = np.zeros((xNumRows,2))
    typeArr[:,vType] = 1
    
    lonLatColList = []
    for start in range(prevTimeStamp):
        lonLatColList.append(seq[start:(-prevTimeStamp+start),1:].copy())
        
    outputLabelLonLat = seq[prevTimeStamp:,1:].copy()
    
    outputLabel = (outputLabelLonLat)
    
    xDataTS = np.zeros((xNumRows,0))
    
    for tS in range(prevTimeStamp):
        xDataTS = np.hstack((xDataTS,lonLatColList[tS]))
        xDataTS = np.hstack((xDataTS,typeArr))
        xDataTS = np.hstack((xDataTS,monData))
    
    return xDataTS, outputLabel

In [9]:
#LAT, LON, SOG, COG, Month (12), Type (2)
tSCol = 18
tSCol = tSCol * prevTS
xDataTS = np.zeros((0,tSCol))
yData = np.zeros((0,4))
for vesselDataSource in vesselDataSources:
    print("Taking data from:%s"%vesselDataSource.srcDir)
    mMSIListFile = vesselDataSource.srcDir + 'TrainTrajCount.txt'
    mMSIList = [line.rstrip('\n') for line in open(mMSIListFile)]
    for traj in mMSIList[0:1]:
        numTraj = traj.split("_")[-1]
        trajAffix = traj[0:-len(numTraj)]
        for num in range(int(numTraj)):
            trajName = trajAffix + str(num) +'.csv'
            rawData = get_traj_data(trajName)
            xTSTemp, yTemp = convert_seq_to_x_y(rawData, vesselDataSource.type, prevTS)
            xDataTS = np.vstack((xDataTS,xTSTemp.copy()))
            yData = np.vstack((yData,yTemp.copy()))

Taking data from:/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/MMSI_1004_TRAIN_INTP/
128
7
Taking data from:/home/jcharla/LiporLab/Data/M122_00_M118_50_33_40_36_40/MMSI_1024_TRAIN_INTP/
210
1


In [10]:
print(xDataTS.shape)
print(yData.shape)

(338, 216)
(338, 4)


In [11]:
xTSToStore = modelDir + "XDataTS.npy"
yToStore = modelDir + "YData.npy"
np.save(xTSToStore, xDataTS)
np.save(yToStore, yData)

In [12]:
numTSFeature = 18
xTSToStore = modelDir + "XDataTS.npy"
yToStore = modelDir + "YData.npy"

xDataTS = np.load(xTSToStore, allow_pickle=True)
yData = np.load(yToStore, allow_pickle=True)

In [13]:
print(xDataTS.shape)
print(yData.shape)

(338, 216)
(338, 4)


In [14]:
sogFirstCol = 2
nextSOGCol = sogFirstCol
sOGMin = np.min(xDataTS[:,nextSOGCol])
sOGMax = np.max(xDataTS[:,nextSOGCol])
while(nextSOGCol < xDataTS.shape[1]):
    tempMinSOG = np.min(xDataTS[:,nextSOGCol])
    tempMaxSOG = np.max(xDataTS[:,nextSOGCol])
    if(tempMinSOG < sOGMin):
        sOGMin = tempMinSOG
        
    if(tempMaxSOG > sOGMax):
        sOGMax = tempMaxSOG
        
    nextSOGCol = nextSOGCol + numTSFeature
print(sOGMin, sOGMax)

10.6 17.4


In [15]:
cogFirstCol = 3
nextCOGCol = cogFirstCol
cOGMin = np.min(xDataTS[:,nextCOGCol])
cOGMax = np.max(xDataTS[:,nextCOGCol])
while(nextCOGCol < xDataTS.shape[1]):
    tempMinCOG = np.min(xDataTS[:,nextCOGCol])
    tempMaxCOG = np.max(xDataTS[:,nextCOGCol])
    if(tempMinCOG < cOGMin):
        cOGMin = tempMinCOG
        
    if(tempMaxCOG > cOGMax):
        cOGMax = tempMaxCOG
        
    nextCOGCol = nextCOGCol + numTSFeature
print(cOGMin, cOGMax)

-151.30285714285714 -79.8855072463768


In [16]:
print(xDataTS[0,:])
print(yData[0,:])
print(xDataTS[1,:])
print(yData[1,:])

[33.71813 -118.50098 11.7 -110.6 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0
 0.0 0.0 0.0 0.0 33.7260747826087 -118.51812347826088 11.72608695652174
 -111.8608695652174 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
 0.0 33.733780163934426 -118.53519327868851 11.6 -112.23934426229508 1.0
 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 33.741399444444454
 -118.55233611111109 11.797222222222222 -110.6 1.0 0.0 0.0 0.0 0.0 0.0 0.0
 1.0 0.0 0.0 0.0 0.0 0.0 0.0 33.74914500000001 -118.56953585714287
 11.727142857142857 -110.32857142857141 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
 0.0 0.0 0.0 0.0 0.0 0.0 33.75685285714286 -118.58660857142857
 11.655714285714286 -112.04285714285713 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
 0.0 0.0 0.0 0.0 0.0 0.0 33.764920000000004 -118.60358 11.6 -109.6 1.0 0.0
 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 33.77299855072464
 -118.62031913043478 11.657971014492755 -110.17971014492753 1.0 0.0 0.0
 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 33.78121260869565
 -118.637267

In [17]:
#Normalise the data
xDataTSNorm = xDataTS.copy()
colAccess = 0
for prevTime in range(prevTS):
    xDataTSNorm[:,(colAccess) + 0] = (xDataTS[:,(colAccess) + 0] - latMin)/(latMax - latMin)
    xDataTSNorm[:,(colAccess) + 1] = (xDataTS[:,(colAccess) + 1] - lonMin)/(lonMax - lonMin)
    xDataTSNorm[:,(colAccess) + 2] = (xDataTS[:,(colAccess) + 2] - sOGMin)/(sOGMax - sOGMin)
    xDataTSNorm[:,(colAccess) + 3] = (xDataTS[:,(colAccess) + 3] - cOGMin)/(cOGMax - cOGMin)
    colAccess = colAccess + numTSFeature

    
xDataTSNorm = np.reshape(xDataTSNorm,(xDataTSNorm.shape[0], prevTS, numTSFeature))

In [18]:
#normalise the output data as well
yLonLatData_0 = (yData[:,0] - latMin)/(latMax - latMin)
yLonLatData_0 = np.reshape(yLonLatData_0,(yLonLatData_0.shape[0],1))
yLonLatData_1 = (yData[:,1] - lonMin)/(lonMax - lonMin)
yLonLatData_1 = np.reshape(yLonLatData_1,(yLonLatData_1.shape[0],1))
yLonLatData_2 = (yData[:,2] - sOGMin)/(sOGMax - sOGMin)
yLonLatData_2 = np.reshape(yLonLatData_2,(yLonLatData_2.shape[0],1))
yLonLatData_3 = (yData[:,3] - cOGMin)/(cOGMax - cOGMin)
yLonLatData_3 = np.reshape(yLonLatData_3,(yLonLatData_3.shape[0],1))
yDataNorm = np.hstack((yLonLatData_0,yLonLatData_1,yLonLatData_2,yLonLatData_3))

In [19]:
from keras import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate

Using TensorFlow backend.
/home/jcharla/anaconda3/envs/conan_pred_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jcharla/anaconda3/envs/conan_pred_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jcharla/anaconda3/envs/conan_pred_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [20]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

W0808 13:47:10.263092 139885656610624 deprecation_wrapper.py:119] From /home/jcharla/anaconda3/envs/conan_pred_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0808 13:47:10.263851 139885656610624 deprecation_wrapper.py:119] From /home/jcharla/anaconda3/envs/conan_pred_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0808 13:47:10.264376 139885656610624 deprecation_wrapper.py:119] From /home/jcharla/anaconda3/envs/conan_pred_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0808 13:47:11.533099 139885656610624 deprecation_wrapper.py:119] From /home/jcharla/anaconda3/envs/conan_pred_env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: T

[]

In [ ]:
# lonLatTS = Input(shape=(prevTS,numTSFeature))
# hidden1 = LSTM(50, return_sequences= True)(lonLatTS)
# hidden21 = LSTM(50, dropout=0.5, recurrent_dropout=0.5)(hidden1)

# hidden22 = LSTM(50, dropout=0.5, recurrent_dropout=0.5)(hidden1)
# hidden23 = LSTM(50, dropout=0.5, recurrent_dropout=0.5)(hidden1)
# hidden24 = LSTM(50, dropout=0.5, recurrent_dropout=0.5)(hidden1)

# lonDense1Drop = Dropout(0.5)(hidden21)
# latDense1Drop = Dropout(0.5)(hidden22)
# sOGDense1Drop = Dropout(0.5)(hidden23)
# cOGDense1Drop = Dropout(0.5)(hidden24)

# lonDense1 = Dense(150, activation='relu')(lonDense1Drop)
# latDense1 = Dense(150, activation='relu')(latDense1Drop)
# sOGDense1 = Dense(150, activation='relu')(sOGDense1Drop)
# cOGDense1 = Dense(150, activation='relu')(cOGDense1Drop)

# lonDense2Drop = Dropout(0.5)(lonDense1)
# latDense2Drop = Dropout(0.5)(latDense1)
# sOGDense2Drop = Dropout(0.5)(sOGDense1)
# cOGDense2Drop = Dropout(0.5)(cOGDense1)

# lonDense2 = Dense(150, activation='relu')(lonDense2Drop)
# latDense2 = Dense(150, activation='relu')(latDense2Drop)
# sOGDense2 = Dense(150, activation='relu')(sOGDense2Drop)
# cOGDense2 = Dense(150, activation='relu')(cOGDense2Drop)

# lonOp = Dense(1, activation='linear')(lonDense2)
# latOp = Dense(1, activation='linear')(latDense2)
# sOGOp = Dense(1, activation='linear')(sOGDense2)
# cOGOp = Dense(1, activation='linear')(cOGDense2)

# modelMultiTask = Model(inputs=lonLatTS, outputs=[lonOp,latOp,sOGOp,cOGOp])

lonLatTS = Input(shape=(prevTS,numTSFeature))
hidden1 = LSTM(50, return_sequences= True)(lonLatTS)
# hidden21 = LSTM(50)(hidden1)
# hidden22 = LSTM(50)(hidden1)
# hidden23 = LSTM(50)(hidden1)
# hidden24 = LSTM(50)(hidden1)
hidden2 = LSTM(50)(hidden1)

lonDense1 = Dense(150, activation='relu')(hidden2)
latDense1 = Dense(150, activation='relu')(hidden2)
sOGDense1 = Dense(150, activation='relu')(hidden2)
cOGDense1 = Dense(150, activation='relu')(hidden2)

lonDense2 = Dense(150, activation='relu')(lonDense1)
latDense2 = Dense(150, activation='relu')(latDense1)
sOGDense2 = Dense(150, activation='relu')(sOGDense1)
cOGDense2 = Dense(150, activation='relu')(cOGDense1)

lonOp = Dense(1, activation='linear')(lonDense2)
latOp = Dense(1, activation='linear')(latDense2)
sOGOp = Dense(1, activation='linear')(sOGDense2)
cOGOp = Dense(1, activation='linear')(cOGDense2)

modelMultiTask = Model(inputs=lonLatTS, outputs=[lonOp,latOp,sOGOp,cOGOp])

In [ ]:
modelMultiTask.summary()

In [ ]:
modelMultiTask.compile(optimizer='adam', loss='mse')

In [ ]:
# modelMultiTaskHist = modelMultiTask.fit(xDataTSNorm, [yDataNorm[:,0],yDataNorm[:,1],yDataNorm[:,2],yDataNorm[:,3]], epochs=50, batch_size = 1024 , verbose = 2)
modelMultiTaskHist = modelMultiTask.fit(xDataTSNorm, [yDataNorm[:,0],yDataNorm[:,1],yDataNorm[:,2],yDataNorm[:,3]], validation_data = [xDataTSTestNorm,[yDataTestNorm[:,0],yDataTestNorm[:,1],yDataTestNorm[:,2],yDataTestNorm[:,3]]], epochs=1000, batch_size = 1024 , verbose = 2)

In [ ]:
plt.plot(modelMultiTaskHist.history['loss'])
plt.plot(modelMultiTaskHist.history['val_loss'])

In [ ]:
modelMultiTaskDir = dataDir + "Model_MULTI_TASK_1000_MSE_WODROP_RANDOM_SAMPLE.h5"

In [ ]:
modelMultiTask.save(modelMultiTaskDir)

In [ ]:
import pickle
with open('trainWODROPRANDOMSAMPLE', 'wb') as file_pi:
    pickle.dump(modelMultiTaskHist.history, file_pi)

In [ ]:
from keras.models import load_model
modelMultiTask = load_model(modelMultiTaskDir)

In [ ]:
def normalize_lon_lat(arr):
    """
    """
    #subtract the minimum 
    #and divide by range
    ret0 = (arr[:,0] - lonMin)/(lonMax - lonMin)
    ret0 = np.reshape(ret0, (ret0.shape[0],1))
    ret1 = (arr[:,1] - latMin)/(latMax - latMin)
    ret1 = np.reshape(ret1, (ret1.shape[0],1))
    ret = np.hstack((ret0, ret1))
    return ret

In [ ]:
#takes 2 time stamps of LON and LAT
#normalises them and 
#and makes the prediction
#de normalize the output
#and return the values
#prevTraj numpy array of 1x2
#currTraj numpy array of 1x2
# def compute_30_min_pred(prevTraj, currTraj, typeVessel):
def compute_30_min_pred(prevTraj, lenVal, yearVal, monVal, destLon, destLat, vType, sOGVal, cOGVal):
    """
    Compute prediction for 30 minutes.
    
    takes LON and LAT of previous time stamps
    normalises them
    and makes prediction
    and returns denormalised LON and LAT values
    """
    #this will be    
    prevTimeStamp = prevTraj.shape[0]
    prevTrajNorm = normalize_lon_lat(prevTraj)
        
    monData = np.zeros((prevTimeStamp, 12))
    monData[:,(12-monVal)] = 1
    
    destArr = np.zeros((prevTimeStamp,2))
    destArr[:,0] = destLon
    destArr[:,1] = destLat

    destArr[:,0] = (destArr[:,0] - lonMin)/(lonMax - lonMin)
    destArr[:,1] = (destArr[:,1] - latMin)/(latMax - latMin)
    
    typeData = np.zeros((prevTimeStamp,2))
    typeData[:,vType] = 1
    
    xTSNorm = np.zeros((prevTimeStamp,0))
    xFNorm = np.zeros((1,0))
    
    sOGValNorm = (sOGVal[:,:] - sOGMin)/(sOGMax - sOGMin)
    cOGValNorm = (cOGVal[:,:] - cOGMin)/(cOGMax - cOGMin)
    
    xTSNorm = np.hstack((xTSNorm,prevTrajNorm))
    xTSNorm = np.hstack((xTSNorm,sOGValNorm))
    xTSNorm = np.hstack((xTSNorm,cOGValNorm))
    xTSNorm = np.hstack((xTSNorm,typeData))
    xTSNorm = np.hstack((xTSNorm,monData))
    xTSNorm = np.hstack((xTSNorm,destArr))

#     xFNorm = np.hstack((xFNorm,typeData))
#     xFNorm = np.hstack((xFNorm,monData))
#     xFNorm = np.hstack((xFNorm,destArr))
    
    xTSNorm = np.reshape(xTSNorm,(1,xTSNorm.shape[0],xTSNorm.shape[1]))
    predLon, predLat, predSOG, predCOG = modelMultiTask.predict(xTSNorm)
    
    #after prediction de normalise it
    predLonScaled = (predLon * (lonMax - lonMin)) + lonMin
    predLatScaled = (predLat * (latMax - latMin)) + latMin
    predSOGScaled = (predSOG * (sOGMax - sOGMin)) + sOGMin
    predCOGScaled = (predCOG * (cOGMax - cOGMin)) + cOGMin
    return predLonScaled, predLatScaled, predSOGScaled, predCOGScaled

In [ ]:
def compute_n_30_min_pred(prevTraj, lenVal, yearVal, monVal, destLon, destLat, vType, sOGVal, cOGVal, n = 1):
    """
    Compute prediction for n*30 minutes.
    
    calls compute_30_min_pred
    according to value of n
    default value of n = 1
    """
    #temp vaariables to store the previous trajectory
    prevTimeStamp = prevTraj.shape[0]
    
    firstLoc = prevTraj.copy()
    firstSOG = sOGVal.copy()
    firstCOG = cOGVal.copy()
    
    #return value will be numpy array of nx2
    ret = np.zeros((0,4))
    for i in range(n):
        #make prediction using 30 min pred
        predLon, predLat, predSOG, predCOG = compute_30_min_pred(firstLoc, lenVal, yearVal, monVal, destLon, destLat, vType, firstSOG, firstCOG)
        #append them in ret
        ret = np.vstack((ret,np.array([[predLon.flatten()[0] \
                                        ,predLat.flatten()[0] \
                                        ,predSOG.flatten()[0] \
                                        ,predCOG.flatten()[0]]])))
        
        #update firstLoc 
        #for next  iteration
        firstLoc = firstLoc[1:,:].copy()
        firstSOG = firstSOG[1:,:].copy()
        firstCOG = firstCOG[1:,:].copy()
        firstLoc = np.vstack((firstLoc,np.array([[predLon.flatten()[0],predLat.flatten()[0]]])))
        firstSOG = np.vstack((firstSOG,np.array([[predSOG.flatten()[0]]])))
        firstCOG = np.vstack((firstCOG,np.array([[predCOG.flatten()[0]]])))
#     return predArr
    return ret

In [ ]:
def get_error_for_traj(srcDir, num, vesselType):
    """
    Compute error for the entire trajectory.
    
    Returns list of error for predictions of upto 
    4 hours i.e. 240 minutes.
    
    Parameters:
        num (int): number of vessel trajectory 
                   whose prediction errors to be computed.
    Returns:
        errorVal (list of floats): list of errors in KM for each
                                   30 min predictions.
    """    
    errorVal = []
    
    vesselTraj,lenData,yearData,monData,destLon,destLat,vType,sOGData,cOGData = get_traj_lon_lat_data_with_len(srcDir, num, vesselType)

    if(vesselTraj.shape[0] < (prevTS + 1)):
        return errorVal
        
        
    predVesselTraj = compute_n_30_min_pred(np.reshape(vesselTraj[0:prevTS,:], (prevTS,2)), lenData, yearData, monData, destLon, destLat, vType \
                                           , np.reshape(sOGData[0:prevTS,:], (prevTS,1))\
                                           , np.reshape(cOGData[0:prevTS,:], (prevTS,1))\
                                           ,n = 8)
    predRange = vesselTraj.shape[0] - prevTS

    #FIXME 8 can come from some variable too
    #for 8 consecutive predictions
    if(predRange > 8):
        predRange = 8

    #use trajectories which has atleast one
    #value to predict
    if(predRange >= 0):
        for pred in range(predRange):    
            trueLon, trueLat = vesselTraj[(prevTS + pred),0], vesselTraj[(prevTS + pred),1]
            predLon, predLat = predVesselTraj[pred,0], predVesselTraj[pred,1]
    #         print(trueLon, trueLat, '-', predLon, predLat)
            errorVal.append(gC.compute_distance(trueLon, trueLat, predLon, predLat))
    return errorVal

In [ ]:
#store errors for all training trajectories
trainDataWholeErrors_1004 = []
# for traj in range(trainTrajNum):
for vesselDataSource in vesselDataSources[0:1]:
    print("Taking data from:%s"%vesselDataSource.srcDir)
    for traj in vesselDataSource.trainList:
        trainDataWholeErrors_1004.append(get_error_for_traj(vesselDataSource.srcDir,traj,vesselDataSource.type))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
trainWholeErr_1004_30 = []
trainWholeErr_1004_60 = []
trainWholeErr_1004_90 = []
trainWholeErr_1004_120 = []
trainWholeErr_1004_150 = []
trainWholeErr_1004_180 = []
trainWholeErr_1004_210 = []
trainWholeErr_1004_240 = []

trainWholeErr_1004_n_30 = [trainWholeErr_1004_30 \
, trainWholeErr_1004_60 \
, trainWholeErr_1004_90 \
, trainWholeErr_1004_120 \
, trainWholeErr_1004_150 \
, trainWholeErr_1004_180 \
, trainWholeErr_1004_210 \
, trainWholeErr_1004_240 \
]

for trajErr in trainDataWholeErrors_1004:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            trainWholeErr_1004_n_30[n_30].append(trajErr[n_30])